# Random Forests for waiting times prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm
from statistics import mean
import math
from statistics import mean
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score, accuracy_score

from sklearn.ensemble import RandomForestClassifier

In [2]:
hp_oHe = pd.read_csv('HP_OHE_3class.csv')
hp_oHe.drop(hp_oHe.tail(17).index,inplace=True) 
hp_oHe = hp_oHe.drop('Unnamed: 0',axis=1)

hp_ME = pd.read_csv("harryPotterClean.csv")
hp_ME.drop(hp_ME.tail(17).index,inplace=True) 
hp_ME = hp_ME.drop('Unnamed: 0',axis=1)

hp_OE = pd.read_csv("harryPotterCleanOE.csv")
hp_OE.drop(hp_OE.tail(17).index,inplace=True) 
hp_OE = hp_OE.drop('Unnamed: 0',axis=1)

In [3]:
def getXandY(df):
    df.drop(df.tail(20).index,inplace=True) 
    x = df.drop(['HP_Forbidden_clean'],axis=1)
    y = df.HP_Forbidden_clean
    return(x,y)

def trainTest(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)
    return(X_train, X_test, y_train, y_test)

#Function to perform oversampling
def overSampling(X_train, y_train, y_test, method):
    X_train_os, y_train_os= method.fit_resample(X_train, y_train)
    # Check the number of records after over sampling
    #print(sorted(Counter(y_train_os).items())) 
    return(X_train_os, y_train_os)

smote = SMOTE(random_state=42)


def testModel(df,var_order,n_vars,n_loops,method):
    highest = 0
    for j in tqdm(range(1,n_vars)):
        #split our dataframe into X and Y
        x,y=getXandY(df)
        #create the lists to store metrics
        acc = []
        rec = []
        preci = []
        f1 = []
        for i in range(n_loops):
            #split the dataFrame into test and train
            X_train, X_test, y_train, y_test = trainTest(x,y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[var_order]
            X_test_r = X_test[var_order]
            df1= X_train_os_r.iloc[:, 0:j] #use only part of the variables
            
            #create and train decision trees
            rnd_clf = RandomForestClassifier(n_jobs=-1)
            rnd_clf.fit(df1, y_train_os)
        
            y_pred=rnd_clf.predict(X_test_r.iloc[:, 0:j])
            ac=metrics.accuracy_score(y_test, y_pred)
            acc.append(ac)
            p=metrics.precision_score(y_test, y_pred,average='macro')
            preci.append(p)
            r=metrics.recall_score(y_test, y_pred,average='macro')
            rec.append(r)
            f=metrics.f1_score(y_test, y_pred, average='macro')
            f1.append(f)
        print(df1.columns)
        print("For {} features: \n Accuracy: {} \n Precision: {} \n Recall: {} \n F1 score: {}".format(
        j,mean(acc),mean(preci),mean(rec),mean(f1)))
        
        if mean(acc)>highest:
            highest = mean(acc)
            best = "best accuracy = {}, with {} features, with {}".format(mean(acc),j,method)
        print(best)
        #print(classification_report(y_test, y_pred))
    print(best)
        
def analizeDF(df,order,n_vars,n_loops):
    for i in range(len(order)):
        print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
        print('The variable order is: \n {}'.format(order[i]))
        testModel(df,order[i],n_vars,n_loops,method[i])
        print('\n \n')

## One hot encoding

In [4]:
#Variable order in one hot encoding:
pear_corrO = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'fog',	'overcast clouds',	'heavy intensity rain',	'minute',	'haze',	'thunderstorm with light rain',	'scattered clouds',	'clear sky',	'mist',	'light intensity drizzle',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'year',	'light rain',	'humidity',	'hour',	'Pandemic']
kend_corrO = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'heavy intensity rain',	'fog',	'year',	'overcast clouds',	'minute',	'haze',	'thunderstorm with light rain',	'mist',	'light intensity drizzle',	'scattered clouds',	'clear sky',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'light rain',	'humidity',	'hour',	'Pandemic']
mutInf_classO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_regO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_class2O = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',	'holiday',	'shower rain',	'light rain',	'thunderstorm',	'fog',	'broken clouds',	'Pandemic',	'thunderstorm with rain',	'light intensity drizzle',	'thunderstorm with light rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds',	'clear sky']
varThreO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',]
mrmrO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds']

orderOHE=[pear_corrO,kend_corrO,mutInf_classO,mutInf_regO,mutInf_class2O,varThreO,mrmrO]

method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'varianceThreshold','MRMR']

In [ ]:
analizeDF(hp_oHe,orderOHE,28,n_loops=20)

  0%|          | 0/27 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'shower rain', 'broken clouds', 'fog', 'overcast clouds', 'heavy intensity rain', 'minute', 'haze', 'thunderstorm with light rain', 'scattered clouds', 'clear sky', 'mist', 'light intensity drizzle', 'few clouds', 'thunderstorm', 'very heavy rain', 'moderate rain', 'thunderstorm with rain', 'year', 'light rain', 'humidity', 'hour', 'Pandemic']


  4%|▎         | 1/27 [00:16<07:07, 16.43s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4388303708580206 
 Precision: 0.3640619531022423 
 Recall: 0.3630652794614665 
 F1 score: 0.362626551943934
best accuracy = 0.4388303708580206, with 1 features, with Pearson_correlation


  7%|▋         | 2/27 [00:34<07:12, 17.29s/it]

...

Index(['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek'], dtype='object')
For 6 features: 
 Accuracy: 0.8539761431411531 
 Precision: 0.8314111216531482 
 Recall: 0.83624241608572 
 F1 score: 0.8336356497887573
best accuracy = 0.8539761431411531, with 6 features, with Pearson_correlation


 26%|██▌       | 7/27 [01:48<04:55, 14.76s/it]

Index(['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek',
       'shower rain'],
      dtype='object')
For 7 features: 
 Accuracy: 0.8552643220526432 
 Precision: 0.8342444525782784 
 Recall: 0.8383590504095191 
 F1 score: 0.8361806673622841
best accuracy = 0.8552643220526432, with 7 features, with Pearson_correlation


 30%|██▉       | 8/27 [02:01<04:32, 14.34s/it]

Index(['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek',
       'shower rain', 'broken clouds'],
      dtype='object')
For 8 features: 
 Accuracy: 0.8599557032115172 
 Precision: 0.8392611967142376 
 Recall: 0.8435270073621295 
 F1 score: 0.8412507145268631
best accuracy = 0.8599557032115172, with 8 features, with Pearson_correlation


## Manual Encoding

In [4]:
#Replace times ending in 5 and also compacting the rest to have only 6 classes
b=hp_ME.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df3=pd.DataFrame(b)
df3.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis3=pd.concat([hp_ME, df3], axis=1)
hp3 = hp_bis3.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)
hp3 = hp3[hp3.HP_Forbidden_clean != 0] #delete rows with 0 min
hp3.HP_Forbidden_clean.unique()

array([2., 1., 3.])

In [5]:
HP_ME = hp3

pear_corr = ['temperature','holiday','day','month','pressure','dayOfTheWeek',
            'report','minute','year','humidity','hour',	'Pandemic']
kend_corr = ['temperature','holiday','month','day','pressure','dayOfTheWeek','report','year',
             'minute','humidity','hour','Pandemic']
mutInf_class = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_reg = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_class2 = ['day','temperature','month','humidity','hour','dayOfTheWeek','pressure',
                 'holiday','year','report','minute','Pandemic']
varThre = ['month','day','year','hour','minute','holiday','dayOfTheWeek',
           'temperature','humidity','pressure','report']
mrmr = ['temperature','dayOfTheWeek','hour','holiday','humidity','day',
        'year','month','pressure','minute','report']
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']

orderME=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

In [6]:
analizeDF(HP_ME,orderME,13,n_loops=20)

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:23<04:15, 23.18s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4362116402116402 
 Precision: 0.3647895344464025 
 Recall: 0.3635805862106963 
 F1 score: 0.3627222224029266
best accuracy = 0.4362116402116402, with 1 features, with Pearson_correlation


 17%|█▋        | 2/12 [00:34<02:44, 16.42s/it]

Index(['temperature', 'holiday'], dtype='object')
For 2 features: 
 Accuracy: 0.456198347107438 
 Precision: 0.4007016661013192 
 Recall: 0.39683896265442575 
 F1 score: 0.39810173228815715
best accuracy = 0.456198347107438, with 2 features, with Pearson_correlation


...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.8783085633404105 
 Precision: 0.8578847254133398 
 Recall: 0.8590234332455967 
 F1 score: 0.8583651599261591
best accuracy = 0.8802355712603063, with 9 features, with MRMR


 92%|█████████▏| 11/12 [01:38<00:08,  8.95s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.8788684148358138 
 Precision: 0.8600673581578482 
 Recall: 0.8596033611382311 
 F1 score: 0.8597286655505955
best accuracy = 0.8802355712603063, with 9 features, with MRMR


100%|██████████| 12/12 [01:47<00:00,  8.97s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.880281523539153 
 Precision: 0.8618164221937163 
 Recall: 0.862273152163532 
 F1 score: 0.861990059918926
best accuracy = 0.880281523539153, with 12 features, with MRMR
best accuracy = 0.880281523539153, with 12 features, with MRMR

 



## Ordinal Encoding

In [7]:
hp_OE.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)

#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_OE.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df1=pd.DataFrame(a)
df1.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis=pd.concat([hp_OE, df1], axis=1)
hp2 = hp_bis.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp2 = hp2[hp2.HP_Forbidden_clean != 0] #delete rows with 0 min
hp2.HP_Forbidden_clean.unique()

#Variable order in ordinal encoding:
pear_corrOE = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']
kend_corrOE = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'report',	'year',	'minute',	'humidity',	'hour',	'Pandemic',]
mutInf_classOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_regOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_class2OE = ['day',	'temperature',	'month',	'humidity',	'hour',	'dayOfTheWeek',	'pressure',	'holiday',	'year',	'report',	'minute',	'Pandemic',]
varThreOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'report']
mrmrOE = ['temperature',	'dayOfTheWeek',	'hour',	'holiday',	'humidity',	'day',	'year',	'month',	'pressure',	'minute',	'report']

orderOE=[pear_corrOE,kend_corrOE,mutInf_classOE,mutInf_regOE,mutInf_class2OE,varThreOE,mrmrOE]

In [8]:
analizeDF(hp2,orderOE,13,n_loops=20)

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:11<02:02, 11.10s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4384021164021164 
 Precision: 0.366707419951909 
 Recall: 0.36534282248213307 
 F1 score: 0.3644074969353454
best accuracy = 0.4384021164021164, with 1 features, with Pearson_correlation


 17%|█▋        | 2/12 [00:21<01:49, 10.96s/it]

Index(['temperature', 'holiday'], dtype='object')
For 2 features: 
 Accuracy: 0.4570989616444162 
 Precision: 0.40138446872133343 
 Recall: 0.3972324314597244 
 F1 score: 0.3985130593488181
best accuracy = 0.4570989616444162, with 2 features, with Pearson_correlation


...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.8796296296296297 
 Precision: 0.860147647323601 
 Recall: 0.8612152685646522 
 F1 score: 0.8606419096240109
best accuracy = 0.8812014134275619, with 9 features, with MRMR


 92%|█████████▏| 11/12 [01:37<00:09,  9.12s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.8809118828254193 
 Precision: 0.8626148548618935 
 Recall: 0.8618620810710773 
 F1 score: 0.8621503710615337
best accuracy = 0.8812014134275619, with 9 features, with MRMR


100%|██████████| 12/12 [01:47<00:00,  8.94s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.8807546723444524 
 Precision: 0.8626961521262452 
 Recall: 0.861842549531403 
 F1 score: 0.8621867216836552
best accuracy = 0.8812014134275619, with 9 features, with MRMR
best accuracy = 0.8812014134275619, with 9 features, with MRMR

 

